In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
print(tf.__version__)

2.8.0


In [5]:
# import splitfolders
# input_folder = "Garbage_Collective_Data/"
# splitfolders.ratio(input_folder, output = "garbage_class",
#                    seed = 0,ratio = (0.70,0.20,0.10),
#                    group_prefix = None )

In [6]:
img_size = [400,400]

train_path = 'garbage_class/train'
valid_path = 'garbage_class/val'
test_path = 'garbage_class/test'

In [7]:
# y_pred = model.predict(val_set)

In [8]:
from tensorflow.keras.layers import MaxPooling2D

In [9]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation ="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=256,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dense(12,activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 16)      208       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        8256      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 28, 28, 64)       0

In [10]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 10911 images belonging to 12 classes.


In [13]:
training_set

In [14]:
val_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3116 images belonging to 12 classes.


In [15]:
val_set

In [16]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1569 images belonging to 12 classes.


In [17]:
print(val_set.class_indices)

{'battery': 0, 'biological': 1, 'brown_glass': 2, 'cardboard': 3, 'clothes': 4, 'green_glass': 5, 'metal': 6, 'paper': 7, 'plastic': 8, 'shoes': 9, 'trash': 10, 'white_glass': 11}


In [20]:
cnn_model = model.fit(
  training_set,
  validation_data=val_set,
  epochs=1,
  steps_per_epoch=len(training_set),
  validation_steps=len(val_set)
)

341/341 [==============================] - 397s 1s/step - loss: 0.4673 - accuracy: 0.8436 - val_loss: 0.7349 - val_accuracy: 0.7770


In [ ]:
model.evaluate(test_set)

In [ ]:
# plot the loss
plt.plot(cnn_model.history['loss'], label='train loss')
plt.plot(cnn_model.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')


# plot the accuracy
plt.plot(cnn_model.history['accuracy'], label='train acc')
plt.plot(cnn_model.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [21]:
y_pred = model.predict(test_set)

In [22]:
y_pred

array([[3.54056537e-01, 1.91570329e-03, 1.22033758e-04, ...,
        2.62807636e-03, 1.83656007e-01, 3.40135276e-01],
       [1.55460224e-01, 1.48912128e-02, 2.73597315e-02, ...,
        2.27141147e-03, 9.24384745e-04, 2.68405471e-02],
       [4.21448112e-06, 1.37220384e-04, 2.10995612e-08, ...,
        5.35007217e-04, 1.36351655e-05, 2.75464595e-06],
       ...,
       [1.57955840e-01, 1.25251466e-03, 1.26426705e-04, ...,
        7.19557166e-01, 1.13316655e-05, 1.18018892e-04],
       [1.14498485e-03, 4.59776324e-08, 2.14731913e-10, ...,
        5.42112030e-05, 1.09794472e-10, 2.16687980e-07],
       [1.01066571e-05, 2.53082305e-07, 1.62145386e-06, ...,
        5.66623748e-05, 1.13823035e-06, 4.07654268e-04]], dtype=float32)

In [ ]:
import numpy as np
pred = np.argmax(y_pred, axis=1)

In [ ]:
# for i,j in zip(list(pred),list(test_set.classes)):
#   print(i,j)

In [ ]:
# print(list(test_set.classes))

In [ ]:
from tensorflow.keras.models import load_model

model.save('model_cnn.h5')

In [28]:
img=image.load_img("C:/Users/Anubhav/Downloads/istockphoto-538807106-1024x1024.jpg",target_size=(224,224))

In [29]:
x=image.img_to_array(img)
x

array([[[178., 167., 111.],
        [187., 168., 138.],
        [255., 255., 239.],
        ...,
        [132., 139., 145.],
        [185., 195., 204.],
        [233., 241., 244.]],

       [[163., 153., 100.],
        [150., 133., 105.],
        [255., 255., 240.],
        ...,
        [194., 192., 193.],
        [124., 136., 150.],
        [142., 158., 171.]],

       [[145., 138.,  92.],
        [164., 148., 122.],
        [252., 254., 240.],
        ...,
        [203., 206., 213.],
        [193., 201., 214.],
        [147., 156., 165.]],

       ...,

       [[219., 223., 224.],
        [221., 225., 226.],
        [200., 204., 205.],
        ...,
        [181., 183., 180.],
        [169., 171., 168.],
        [154., 156., 153.]],

       [[206., 210., 211.],
        [211., 215., 216.],
        [199., 203., 204.],
        ...,
        [175., 177., 176.],
        [156., 158., 157.],
        [163., 165., 164.]],

       [[197., 198., 200.],
        [203., 207., 208.],
        [185., 1

In [ ]:
x.shape

In [32]:
x=x/255

In [33]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [34]:
x=np.expand_dims(x,axis=0)
# # img_data=preprocess_input(x)
x.shape

(1, 224, 224, 3)

In [40]:
preds = model.predict(x)

In [41]:
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

In [43]:
#pred_class = preds.argmax(axis=-1)            # Simple argmax
pred_class = decode_predictions(preds, top=1)   # ImageNet Decode
# result = str(pred_class[0][0][1])

ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (1, 12)

In [44]:

a = np.argmax(model.predict(x), axis=1)

In [ ]:
if(a==0):
  print("battery")
elif(a==1):
  print("biological")
elif(a==2):
  print("brown_glass")
elif(a==3):
  print("cardboard")
elif(a==4):
  print("clothes")
elif(a==5):
  print("green_glass")
elif(a==6):
  print("metal")
elif(a==7):
  print("paper")
elif(a==8):
  print("plastic")
elif(a==9):
  print("shoes")
elif(a==10):
  print("trash")
elif(a==11):
  print("white_glass")